In [1]:
import os
import logging
import numpy as np
from datetime import datetime

from pinf.analytical_field import get_analytic_b_field
from pinf.performance_metrics import metrics
from pinf.trainer import NF2Trainer
from pinf.unpack import load_cube

In [2]:
b = get_analytic_b_field(n=1, m=1, l=0.3, psi=np.pi/2, resolution=64, bounds=[-1, 1, -1, 1, 0, 2])

In [3]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "0"

base_path = './run2'
meta_path = None

bin = 1

height = 64
spatial_norm = 32
b_norm = 100

meta_info = None
dim = 256
positional_encoding = False
use_potential_boundary = True
potential_strides = 1
use_vector_potential = False
lambda_div = 1e-1
lambda_ff = 1e-1
decay_iterations = 25000
device = None
work_directory = None

total_iterations = 50000
batch_size = 10000
log_interval = 100
validation_interval = 100
num_workers = 4

# init logging
os.makedirs(base_path, exist_ok=True)
log = logging.getLogger()
log.setLevel(logging.INFO)
for hdlr in log.handlers[:]:  # remove all old handlers
    log.removeHandler(hdlr)
log.addHandler(logging.FileHandler("{0}/{1}.log".format(base_path, "info_log")))  # set the new file handler
log.addHandler(logging.StreamHandler())  # set the new console handler

base_path = os.path.join(base_path, 'dim%d_bin%d_pf%s_ld%s_lf%s' % (
        dim, bin, str(use_potential_boundary), lambda_div, lambda_ff))

b_cube = b[:, :, 0, :]

trainer = NF2Trainer(base_path, b_cube, height, spatial_norm, b_norm, 
                     meta_info=meta_info, dim=dim, positional_encoding=positional_encoding, 
                     meta_path=meta_path, use_potential_boundary=use_potential_boundary, 
                     potential_strides=potential_strides, use_vector_potential=use_vector_potential,
                     lambda_div=lambda_div, lambda_ff=lambda_ff, decay_iterations=decay_iterations,
                     device=device, work_directory=work_directory)

Configuration:
dim: 256, lambda_div: 0.100000, lambda_ff: 0.100000, decay_iterations: 25000, potential: True, vector_potential: False, 
Using device: cuda (gpus 1) ['NVIDIA GeForce RTX 3060']
Potential Boundary: 100%|██████████| 1/1 [00:00<00:00,  1.38it/s]


In [5]:
start_time = datetime.now()
trainer.train(total_iterations, batch_size, 
              log_interval=log_interval, validation_interval=validation_interval, 
              num_workers=num_workers)
log.info(f'TOTAL RUNTIME: {datetime.now() - start_time}')

Training:   0%|          | 0/50000 [00:00<?, ?it/s][Iteration 000001/050000] [BC: 0.00003453; Div: 0.00005190; For: 0.00011409] [0:00:03.710738]
Lambda B: 0.999724
LR: 0.000058
Validation: 100%|██████████| 27/27 [00:00<00:00, 65.39it/s]
Validation [Cube: BC: 0.777; Div: 0.007; For: 0.022; Sig: 3.882]
Training:   0%|          | 37/50000 [00:09<3:34:20,  3.89it/s]


KeyboardInterrupt: 